# Feature selection using Cuckoo Search

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from niapy.problems import Problem
from niapy.task import Task

from niapy.algorithms.basic import CuckooSearch
import pandas as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
class GaussianNBFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(GaussianNB(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [3]:
class LogisticRegressionFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(LogisticRegression(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [4]:
class DecisionTreeClassifierFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(DecisionTreeClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [5]:
class KNeighborsClassifierFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(KNeighborsClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [6]:
class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [7]:
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
feature_names = dataset.feature_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

problem_nb = GaussianNBFeatureSelection(X_train, y_train)
problem_lr = LogisticRegressionFeatureSelection(X_train, y_train)
problem_dt = DecisionTreeClassifierFeatureSelection(X_train, y_train)
problem_knn = KNeighborsClassifierFeatureSelection(X_train, y_train)
problem_svc = SVMFeatureSelection(X_train, y_train)
#task = Task(problem, max_iters=200)

In [8]:
cuckoodict = {}

# Cuckoo Search (1/2: Change to population size)

In [9]:
# Set saved variable for checked parameters
bestScore = 0.0
bestCriterion = None
pop_num = 0
pop_features = None

for x in range(10,80,15):
    # GaussianNB
    task = Task(problem_nb, max_iters=200)
        
    print("-"*50)
    print("Cuckoo search: Population size = "+ str(x))
    algorithm = CuckooSearch(population_size=x, pa=0.25)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = GaussianNB()
    model_all = GaussianNB()
    model_name = "GaussianNB"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)       
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
    # DecisionTreeClassifier
    task = Task(problem_dt, max_iters=200)
        
    print("-"*50)
    print("Cuckoo search: Population size = "+ str(x))
    algorithm = CuckooSearch(population_size=x, pa=0.25)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = DecisionTreeClassifier()
    model_all = DecisionTreeClassifier()
    model_name = "DecisionTreeClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)       
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
    # KNeighborsClassifier
    task = Task(problem_knn, max_iters=200)
        
    print("-"*50)
    print("Cuckoo search: Population size = "+ str(x))
    algorithm = CuckooSearch(population_size=x, pa=0.25)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = KNeighborsClassifier()
    model_all = KNeighborsClassifier()
    model_name = "KNeighborsClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)       
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
    # SVC    
    task = Task(problem_svc, max_iters=200)
        
    print("-"*50)
    print("Cuckoo search: Population size = "+ str(x))
    algorithm = CuckooSearch(population_size=x, pa=0.25)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = SVC()
    model_all = SVC()
    model_name = "SVC"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)       
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))

cuckoodict["Cuckoo-Population"] = "pop: " + str(bestCriterion) + ", accuracy: " + str(round(bestScore, 5))

--------------------------------------------------
Cuckoo search: Population size = 10
Number of selected features: 9
Selected features: mean texture, mean concave points, texture error, smoothness error, symmetry error, worst texture, worst area, worst smoothness, worst concave points
GaussianNBSubset accuracy: 0.956140350877193
GaussianNBAll Features Accuracy: 0.9649122807017544
--------------------------------------------------
Cuckoo search: Population size = 10
Number of selected features: 12
Selected features: mean texture, mean perimeter, mean area, mean compactness, mean fractal dimension, perimeter error, smoothness error, compactness error, concavity error, concave points error, worst area, worst smoothness
DecisionTreeClassifierSubset accuracy: 0.9473684210526315
DecisionTreeClassifierAll Features Accuracy: 0.9473684210526315
--------------------------------------------------
Cuckoo search: Population size = 10
Number of selected features: 7
Selected features: mean radius, m

In [10]:
cuckoodict

{'Cuckoo-Population': 'pop: 70, accuracy: 0.99123'}

# Cuckoo Search (2/2: Change to the probability of nest being abandoned (pa))

In [11]:
# Set saved variable for checked parameters
bestScore = 0.0
bestCriterion = None
pa_num = 0
pa_features = None

for x in range(25, 100, 15):
    # GaussianNB
    task = Task(problem_nb, max_iters=200)
    
    print("-"*50)
    print("Cuckoo search: pa = "+ str(x/100))
    algorithm = CuckooSearch(population_size=25, pa=x/100)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = GaussianNB()
    model_all = GaussianNB()
    model_name = "GaussianNB"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pa_num = selected_features.sum()
        pa_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
    # DecisionTreeClassifier
    task = Task(problem_dt, max_iters=200)
    
    print("-"*50)
    print("Cuckoo search: pa = "+ str(x/100))
    algorithm = CuckooSearch(population_size=25, pa=x/100)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = DecisionTreeClassifier()
    model_all = DecisionTreeClassifier()
    model_name = "DecisionTreeClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pa_num = selected_features.sum()
        pa_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
    # KNeighborsClassifier
    task = Task(problem_knn, max_iters=200)
    
    print("-"*50)
    print("Cuckoo search: pa = "+ str(x/100))
    algorithm = CuckooSearch(population_size=25, pa=x/100)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = KNeighborsClassifier()
    model_all = KNeighborsClassifier()
    model_name = "KNeighborsClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pa_num = selected_features.sum()
        pa_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
    # SVC     
    task = Task(problem_svc, max_iters=200)
    
    print("-"*50)
    print("Cuckoo search: pa = "+ str(x/100))
    algorithm = CuckooSearch(population_size=25, pa=x/100)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = SVC()
    model_all = SVC()
    model_name = "SVC"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + 'Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pa_num = selected_features.sum()
        pa_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + 'All Features Accuracy:', model_all.score(X_test, y_test))
    
cuckoodict["Cuckoo-Pa"] = "pa: " + str(bestCriterion) + ", accuracy: " + str(round(bestScore, 5))

--------------------------------------------------
Cuckoo search: pa = 0.25
Number of selected features: 7
Selected features: mean texture, mean concave points, mean fractal dimension, worst texture, worst area, worst smoothness, worst concave points
GaussianNBSubset accuracy: 0.956140350877193
GaussianNBAll Features Accuracy: 0.9649122807017544
--------------------------------------------------
Cuckoo search: pa = 0.25
Number of selected features: 12
Selected features: mean area, mean compactness, mean concave points, texture error, perimeter error, smoothness error, concave points error, fractal dimension error, worst texture, worst area, worst smoothness, worst concave points
DecisionTreeClassifierSubset accuracy: 0.9473684210526315
DecisionTreeClassifierAll Features Accuracy: 0.956140350877193
--------------------------------------------------
Cuckoo search: pa = 0.25
Number of selected features: 7
Selected features: mean texture, mean concave points, radius error, symmetry error, 

# Data Preprocess

In [12]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['target'] = dataset.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [13]:
cuckoodict

{'Cuckoo-Population': 'pop: 70, accuracy: 0.99123',
 'Cuckoo-Pa': 'pa: 70, accuracy: 0.99123'}

In [14]:
print("pop_num: {}, pop_features: {}\npa_num: {}, pa_features: {}\nn".format(pop_num, pop_features, pa_num, pa_features))

pop_num: 12, pop_features: ['mean texture', 'mean compactness', 'mean concavity', 'mean concave points', 'perimeter error', 'area error', 'compactness error', 'concavity error', 'concave points error', 'worst area', 'worst concavity', 'worst concave points']
pa_num: 13, pa_features: ['mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean concave points', 'radius error', 'perimeter error', 'area error', 'concavity error', 'worst radius', 'worst area', 'worst smoothness', 'worst concave points']
n


In [15]:
newdf = df.drop(columns=[col for col in df if col not in pop_features])
newdf.head()

,mean texture,mean compactness,mean concavity,mean concave points,perimeter error,area error,compactness error,concavity error,concave points error,worst area,worst concavity,worst concave points
0,10.38,0.27760,0.3001,0.14710,8.589,153.40,0.04904,0.05373,0.01587,2019.0,0.7119,0.2654
1,17.77,0.07864,0.0869,0.07017,3.398,74.08,0.01308,0.01860,0.01340,1956.0,0.2416,0.1860
2,21.25,0.15990,0.1974,0.12790,4.585,94.03,0.04006,0.03832,0.02058,1709.0,0.4504,0.2430
3,20.38,0.28390,0.2414,0.10520,3.445,27.23,0.07458,0.05661,0.01867,567.7,0.6869,0.2575
4,14.34,0.13280,0.1980,0.10430,5.438,94.44,0.02461,0.05688,0.01885,1575.0,0.4000,0.1625


In [16]:
y =  dataset.target
X = newdf
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 21)

#Feature Scaling, this is important as itt will allow the algorithm to quickly learn a better solution.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(569, 12) (569,)


# Naive Bayes Model

In [17]:
def gaussianNB():
    
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train,y_train)
    y_pred_test = gnb.predict(X_test)
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_test,y_pred_test)
    print("Naive Bayes : ", acc)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred_test, target_names=b_class))
    
    return y_pred_test

# LR Model

In [18]:
def logisticRegression():
    from sklearn.linear_model import LogisticRegression
    logreg = LogisticRegression(solver = 'liblinear',multi_class='auto')
    logreg.fit(X_train,y_train)
    y_pred = logreg.predict(X_test)
    from sklearn.metrics import accuracy_score
    acc1 = accuracy_score(y_test,y_pred)
    print("Logistic Regression: ", acc1)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred, target_names=b_class))
    
    return y_pred

# Decision Tree Model

In [19]:
def decisionTrees():
    from sklearn.tree import DecisionTreeClassifier
    dt = DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    y_pred2 = dt.predict(X_test)
    acc2 = accuracy_score(y_test,y_pred2)
    print(acc2)
    print("Decision Trees:", acc2)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred2, target_names=b_class))

    return y_pred2

# KNN Model

In [20]:
def knn():
    
    from sklearn.neighbors import KNeighborsClassifier
    clf = KNeighborsClassifier(n_neighbors=8,algorithm='ball_tree')
    clf.fit(X_train,y_train)
    y_pred3 = clf.predict(X_test)
    acc3 =   accuracy_score(y_test,y_pred3)
    print("K Nearest Neighbors: ", acc3)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred3, target_names=b_class))  
    
    return y_pred3

# Support Vector Model

In [21]:
def svc():
    
    from sklearn.svm import SVC
    svc1 = SVC(C=50,kernel='rbf',gamma=1)     
    svc1.fit(X_train,y_train)
    y_pred4 = svc1.predict(X_test)
    from sklearn.metrics import accuracy_score
    acc4=    accuracy_score(y_test,y_pred4)
    print("Support Vector Classifier Accuracy", acc4)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred4, target_names=b_class))  
    
    return y_pred4

# Model Results

In [22]:
models = {
        "gaussianNB": gaussianNB(),
        "decisionTrees": decisionTrees() ,
        "logisticRegression": logisticRegression(),
        "svc": svc(),
        "knn": knn()
    }

results = {}
print("\nNow testing algorithms")
for algo in models:

    print("trained model: ", algo )

print("\n")

Naive Bayes :  0.9298245614035088
              precision    recall  f1-score   support

      benign       0.90      0.90      0.90        39
      malign       0.95      0.95      0.95        75

    accuracy                           0.93       114
   macro avg       0.92      0.92      0.92       114
weighted avg       0.93      0.93      0.93       114

0.9035087719298246
Decision Trees: 0.9035087719298246
              precision    recall  f1-score   support

      benign       0.89      0.82      0.85        39
      malign       0.91      0.95      0.93        75

    accuracy                           0.90       114
   macro avg       0.90      0.88      0.89       114
weighted avg       0.90      0.90      0.90       114

Logistic Regression:  0.9912280701754386
              precision    recall  f1-score   support

      benign       1.00      0.97      0.99        39
      malign       0.99      1.00      0.99        75

    accuracy                           0.99       114

In [23]:
cuckoodict

{'Cuckoo-Population': 'pop: 70, accuracy: 0.99123',
 'Cuckoo-Pa': 'pa: 70, accuracy: 0.99123'}